In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import os
import shutil
directory_out = './file/4.compatibility'
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

directory_in = './file/3.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                           header=0,
                                                                           dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input = df_input[df_input['Vehicle Url'] != ''].reset_index(drop=True)
    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_index, crawler_series = crawler_tuple

        # = = =  = = =  = = =

        try:
            list_dict_correct_temp, page = [], 0
            while True:

                crawler_retry = 0
                while True:
                    crawler_retry += 1

                    try:
                        url_request = f'''{crawler_series['Vehicle Url']}&num=50&start={page * 50}'''

                        resp = requests.get(url_request,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(5, 15))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))

                            if html.xpath('//div[@class="detail-app-pager"]'):
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = =  = = =  = = =

                next_ = html.xpath('//a[@id="pagingBottom_nextButton"]')
                if page == 0 and next_ and next_[0].xpath('./@href')[0].strip() == 'Search/#':
                    raise

                # = = =  = = =  = = =

                list_text, list_tr = [text.strip() for text in html.xpath('//th[@scope="col"]/text()')], html.xpath('//tr[@class="detail-app-row"]')

                # = = =  = = =  = = =

                list_dict_correct_temp_temp = [{'Vehicle Url': crawler_series['Vehicle Url'],
                                                'Page': page + 1,
                                                'Row': i + 1} for i in range(len(list_tr))]

                for i, tr in enumerate(list_tr):
                    for j, text in enumerate(list_text):
                        list_dict_correct_temp_temp[i][text] = tr.xpath(f'./td[{j+1}]/text()')[0].strip()

                list_dict_correct_temp.extend(list_dict_correct_temp_temp)

                # = = =  = = =  = = =

                if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
                    clear_output()
                print(f'''[第{page + 1}页] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

                # = = =  = = =  = = =

                if not next_:
                    break

                # = = =  = = =  = = =

                page += 1

            # = = =  = = =  = = =

            if not list_dict_correct_temp:
                raise

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Vehicle Url', 'Page', 'Row'],
                                                                 ascending=[True, True, True],
                                                                 ignore_index=True)
        df_correct['Year'] = df_correct['Year'].astype(int)
        df_correct.to_excel(os.path.join(directory_out, f'''{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-compatibility_error.xlsx''', index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-compatibility.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[第1页] - [请求31次] - [剩余29条] - [1（100.00%）] - [2024/12/13 09:44:54] - 115. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=154567&PartType=Timing%20Cover&num=50&start=0

[第1页] - [请求8次] - [剩余28条] - [1（100.00%）] - [2024/12/13 09:44:54] - 162. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=137282&PartType=Timing%20Cover&num=50&start=0

[第9页] - [请求25次] - [剩余27条] - [1（100.00%）] - [2024/12/13 09:44:54] - 63. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=22413&PartType=Timing%20Cover&num=50&start=400

[第1页] - [请求6次] - [剩余27条] - [1（100.00%）] - [2024/12/13 09:44:55] - 143. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=48429&PartType=Timing%20Cover&num=50&start=0

[第1页] - [请求3次] - [剩余26条] - [1（100.00%）] - [2024/12/13 09:44:55] - 163. https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84483&PartType=Timing%20Tab&num=50&start=0

[第5页] - [请求11次] - [剩余26条] - [1（100.00%）] - [2024/12/13 09:44:55] - 116. https://www.dormanproducts.com/itemdetai